Last time I explored how to index a data set I created exploring three different methodologies for creating stats for 5E D&D characters.  In this blog post, I'm going to run some descriptive statistics as well as some exploratory data analysis.

In [1]:
#import functions
import pandas as pd
import matplotlib.pyplot as plt

#for in notebook graphic exploration
%matplotlib inline

In [2]:
#read in the data
df = pd.read_csv('1000CharSimulated20seed.csv',index_col=0)

In [4]:
#use the shape function to look at the data
df.shape

#shape gives us the number of rows and the number of columns
#unlike head() or tail(), shape is an attribute of the data frame, not a method 

(3000, 13)

In [5]:
#use describe to get some high level statistics
df.describe()

,strength,dexterity,constitution,intellegence,wisdom,charisma,str mod,dex mod,con mod,int mod,wis mod,char mod
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,11.915333,11.990333,12.007333,12.015000,11.947000,11.866333,0.701667,0.737667,0.751667,0.754333,0.730000,0.676000
std,3.074191,3.187417,3.112851,3.126665,3.150794,3.142784,1.547728,1.616494,1.575275,1.582872,1.593303,1.589927
min,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000
25%,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,14.000000,15.000000,14.000000,15.000000,14.000000,14.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


The describe function gives us a lot of great information on each of the variables including:
* Count
* Mean
* Standard Deviation
* Whisker PLot Inputs (min, 25th percentile 50th percentile, 75th percentile, and max)

Notice how describe doesn't provide any summary statistics for the qualitative roll_type variable.

While this is useful, it is not quite fair to summarize all the data together because it represents the aggregation of three different methodologies for creating characters.  So what I really need is a way to run describe() three different times, one for each methodology. Luckily, I can do that with groupby

In [25]:
include_list = ['mean','std','25%','50%','75%']
df.groupby('roll_type').describe()

char mod                                           charisma  \
              count   mean       std  min  25%  50%  75%  max    count   
roll_type                                                                
3D6          1000.0 -0.088  1.540334 -4.0 -1.0  0.0  1.0  4.0   1000.0   
4D6DropLow   1000.0  0.825  1.448611 -4.0  0.0  1.0  2.0  4.0   1000.0   
Colville     1000.0  1.291  1.460292 -3.0  0.0  1.0  2.0  4.0   1000.0   

                    ...  wis mod       wisdom                               \
              mean  ...      75%  max   count    mean       std  min   25%   
roll_type           ...                                                      
3D6         10.361  ...      1.0  4.0  1000.0  10.608  3.079581  3.0   8.0   
4D6DropLow  12.139  ...      2.0  4.0  1000.0  12.089  2.828972  3.0  10.0   
Colville    13.099  ...      2.0  4.0  1000.0  13.144  3.011035  3.0  11.0   

                              
             50%   75%   max  
roll_type                     
3D6         11.0  13.0  18.0  
4D6DropLow  12.0  14.0  18.0  
Colville    14.0  15.0  18.0  

[3 rows x 96 columns]

This is exactly what we want, but it is a little hard to see.  So let's try and modify the code so we get rid of stuff we don't need and modifying it so it is easier to read.

Information we don't need:
* Count (it's 3000 for all of them)
* Min (min is 3 for abilities and -4 for modifiers.  The results of rolling 3 ones.  Ouch!)
* Max (max is the opposite of min.  18 for abilities and + for modifiers.  The results of rolling 3 sixes.  Much better)

In [32]:
df.groupby('roll_type').describe().unstack()

                 roll_type 
char mod  count  3D6           1000.000000
                 4D6DropLow    1000.000000
                 Colville      1000.000000
          mean   3D6             -0.088000
                 4D6DropLow       0.825000
                 Colville         1.291000
          std    3D6              1.540334
                 4D6DropLow       1.448611
                 Colville         1.460292
          min    3D6             -4.000000
                 4D6DropLow      -4.000000
                 Colville        -3.000000
          25%    3D6             -1.000000
                 4D6DropLow       0.000000
                 Colville         0.000000
          50%    3D6              0.000000
                 4D6DropLow       1.000000
                 Colville         1.000000
          75%    3D6              1.000000
                 4D6DropLow       2.000000
                 Colville         2.000000
          max    3D6              4.000000
                 4D6DropLo

That's a little better, but still a little tricky to read.  Instead let's use what we learned from indexing last time to look at specific data in more detail.  Going with my theme of bards, let's look at intellegence and charisma.

In [56]:
df.groupby('roll_type').describe().loc[:,["intellegence","charisma"]]

charisma                                                 \
              count    mean       std  min   25%   50%   75%   max   
roll_type                                                            
3D6          1000.0  10.361  3.028506  3.0   8.0  10.0  13.0  18.0   
4D6DropLow   1000.0  12.139  2.861095  3.0  10.0  12.0  14.0  18.0   
Colville     1000.0  13.099  2.903382  5.0  11.0  13.0  15.0  18.0   

           intellegence                                                 
                  count    mean       std  min   25%   50%   75%   max  
roll_type                                                               
3D6              1000.0  10.554  2.956658  3.0   9.0  11.0  13.0  18.0  
4D6DropLow       1000.0  12.528  2.835711  3.0  11.0  13.0  15.0  18.0  
Colville         1000.0  12.963  3.043829  4.0  11.0  13.0  15.0  18.0